In [1]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
     |████████████████████████████████| 16.6 MB 47.1 MB/s 
     |████████████████████████████████| 7.8 MB 61.8 MB/s 
     |████████████████████████████████| 1.6 MB 50.2 MB/s 
     |████████████████████████████████| 128 kB 61.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 132 kB 47.2 MB/s 


In [ ]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'TX',
    root_path = root_path,
)

refresh = [
    'acs5',
    'assignments',
    'crosswalks',
    'elections',
    'final',
    'geo',
    'pl',
    'plans',
    # 'shapes',
    'transformers',
]

for ds in self.bq.client.list_datasets():
    if ds.dataset_id in refresh:
        self.bq.del_ds(ds)

# level='vtd'
# year = 2018
# self.get_shapes(overwrite=True)
# self.get_pl(overwrite=True)
# self.get_assignments(overwrite=True)
# self.get_plans(overwrite=True)
# self.get_geo(overwrite=True)
# self.get_crosswalks(overwrite=True)
# self.get_transformer(overwrite=True)
# self.get_acs5(rear=year, ov/erwrite=True)
# self.transform_acs5(level=level, year=year, extra_cols=('geometry'))#, overwrite=True
# df = self.get_elections()
# self.get_geo(level='vtd')
self.get_final()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
creating elections.TX_vtd2022_raw ... down
downloading from https://data.capitol.texas.gov/dataset/35b16aee-0bb0-4866-b1ec-859f1f044241/resource/5af9f5e2-ca14-4e5d-880e-3c3cd891d3ed/download/2020-general-vtd-election-data-2020.zip ... unzipping


In [2]:
# X = 
# df.to_list(orient='index')
# ut.listify(df)
# for camp, cand in df.set_index('campaign').iterrows():
# # for k, v, in X:
#     print(camp, cand)
#     print(type(camp), cand)
72056 / 9007

8.0

In [7]:
df.groupby(['office', 'year']).to_dict(orient='index')
# df.to_dict?(orient=')
# df.to_dict(orient='index')

AttributeError: ignored

In [2]:
d = dict()
for c in campaigns:
    print(c)

USSen_2018
President_2020
USSen_2020
Comptroller_2018
President_2016
Governor_2018
AttorneyGen_2018
LtGovernor_2018


In [7]:
import geopandas as gpd
qry = 'select * from redistricting-361203.acs5.TX_tract2018_vtd2020 limit 5'
df = self.bq.qry_to_df(qry).fillna(0)

# res = self.bq.run_qry(qry)#.fillna(0)
# df = ut.prep(res.to_dataframe())
# print(df.dtypes)
# geo = gpd.GeoSeries.from_wkt(df['geometry'].astype(str), crs=CRS['bigquery'])
# print(df['geometry'].astype(str).dtype)
# geo

In [3]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   year                               10 non-null     Int64   
 1   vtd2020                            10 non-null     Int64   
 2   all_tot_pop                        10 non-null     Float64 
 3   white_tot_pop                      10 non-null     Float64 
 4   hisp_tot_pop                       10 non-null     Float64 
 5   all_vap_pop                        10 non-null     Float64 
 6   white_vap_pop                      10 non-null     Float64 
 7   hisp_vap_pop                       10 non-null     Float64 
 8   all_vap_elderly                    10 non-null     Float64 
 9   white_vap_elderly                  10 non-null     Float64 
 10  hisp_vap_elderly                   10 non-null     Float64 
 11  all_vap_poverty                    10 non-nul

In [ ]:
# ut.listify(df)
# df.squeeze().tolist()#.values##.values
# df.values
campaigns = tuple(campaigns)
election='2020_general_President'
year, election, office = election.split('_')
geoid = 'vtd2020'
qry = f"""
select {geoid}, campaign, votes
from {self.get_elections()}
--where year={year} and election={election} and office={office} and party in ('D', 'R')
where campaign in {campaigns}
pivot(sum(votes) for campaign in {campaigns})
"""
print(qry)


select vtd2020, campaign, votes
from elections.TX_vtd2022
--where year=2020 and election=general and office=President and party in ('D', 'R')
where campaign in ('2020_general_President_Biden_D', '2020_general_President_Trump_R')
pivot(sum(votes) for campaign in ('2020_general_President_Biden_D', '2020_general_President_Trump_R'))



In [ ]:
select * except (geometry), geometry,
from (
    select
        *,
        st_perimeter(geometry) as perim,
        case when perim < 0.1 then 0 else 4 * 3.141592653589793 * atot / (perim * perim) end as polsby_popper,
    from (
        select
            vtd2020,
            min(dist_to_border) as dist_to_border,
            sum(aland) as aland,
            sum(awater) as awater,
            sum(atot) as atot,
            st_union_agg(geometry) as geometry,
        from geo.RI_block2020
        group by vtd2020
    ) as S
join (
    select vtd2020, sum(all_tot_pop) as all_tot_pop, sum(all_vap_pop) as all_vap_pop, sum(white_tot_pop) as white_tot_pop, sum(white_vap_pop) as white_vap_pop, sum(hisp_tot_pop) as hisp_tot_pop, sum(hisp_vap_pop) as hisp_vap_pop, sum(other_tot_pop) as other_tot_pop, sum(other_vap_pop) as other_vap_pop,
    from geo.RI_block2020
    group by vtd2020
) as P
using (vtd2020)
join (
    select vtd2020, county2020, congress2010, senate2010, house2010
    from geo.RI_block2020
    qualify row_number() over (partition by vtd2020 order by all_tot_pop desc) = 1
) as D
using (vtd2020)

'02'

In [ ]:
from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp
geoid='block2020'
path = pathlib.Path('/content/drive/MyDrive/gerrymandering/2022-12/data/plans/')
# pd.read_csv(f, names=['block2020', 'hi'], header=0)
L = [ut.prep(pd.read_csv(file, names=[geoid, file.stem], header=0)).set_index(geoid) for file in path.iterdir() if file.suffix == '.csv']

In [ ]:
df = pd.concat(L, axis=1)

In [ ]:
df.nunique(axis=0)

plans2100    31
plans2101    31
plans2103     3
plans2108    14
plans2109    31
             ..
planc2185     3
planc2186    38
planc2193    38
planc2194    38
planc2195    17
Length: 242, dtype: int64

In [ ]:
T = [
    # 'redistricting-361203.shapes.RI_block2020',
    # 'redistricting-361203.pl.RI_block2020',
    'redistricting-361203.assignments.RI_block2020',
    'redistricting-361203.geo.RI_block2020',
    # 'redistricting-361203.crosswalks.RI_block2020_raw',
    'redistricting-361203.crosswalks.RI_block2020',
    'redistricting-361203.transformers.RI_tract2018',
]
x = 'vtd2020'
for tbl in T:
    
    qry = f'select count(*) from (select distinct {x} FROM {tbl})'
    print(qry)
    print(self.bq.qry_to_df(qry).squeeze())
    

select count(*) from (select distinct vtd2020 FROM redistricting-361203.assignments.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.geo.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.crosswalks.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.transformers.RI_tract2018)
423


In [ ]:
vp.unzipper('/home/content/sample_data/mnist_test.csv')

In [ ]:
refresh = [
    # 'shapes',
    # 'pl',
    # 'assignments',
    'geo',
    'crosswalks',
    'transformers',
    'acs5',
]

for ds in self.bq.client.list_datasets():
    # if ds.dataset_id in refresh:
    print(ds.dataset_id, ds.dataset_id in refresh)
        # self.bq.del_ds(ds)


acs5 True
assignments False
crosswalks True
pl False
shapes False
transformers True


In [ ]:
subpops = {
    'all_tot_pop'  : 'p2_001n',
    'all_vap_pop'  : 'p4_001n',
    'wite_tot_pop' : 'p2_005n',
    'white_vap_pop': 'p4_005n',
    'hisp_tot_pop' : 'p2_002n',
    'hisp_vap_pop' : 'p4_002n',
    'other_tot_pop': None,
    'other_vap_pop': None,
}
{v:k for k,v in subpops.items()}
# dict(zip(d.items()))

{'p2_001n': 'all_tot_pop',
 'p4_001n': 'all_vap_pop',
 'p2_005n': 'wite_tot_pop',
 'p4_005n': 'white_vap_pop',
 'p2_002n': 'hisp_tot_pop',
 'p4_002n': 'hisp_vap_pop',
 None: 'other_vap_pop'}

In [ ]:
x = 'ababab'
x[:x.rfind('a')]

'abab'